# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
It's a classification problem: the answer is boolean. Either a student need intervention or not. 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
%matplotlib inline
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

/Users/nathanzylbersztejn/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features =  student_data.shape[1]-1

# TODO: Calculate passing students
n_passed = student_data[student_data['passed'] == 'yes'].shape[0]

# TODO: Calculate failing students
n_failed = student_data[student_data['passed'] == 'no'].shape[0]

# TODO: Calculate graduation rate
grad_rate = round(100 * float(n_passed) / (n_passed + n_failed),0)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.0f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))
X_all.to_csv("processed_data.csv", sep='\t', encoding='utf-8')

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300
# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=95, random_state=0)


# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **

**Naive Bayes**

_General applications:_

Many text classification problems, such as spam detection, sentiment analysis, or content categorization (a new article is about sports or business).
_Advantages:_
- Naive bayes is a fast and easy to configure algorithm since you only have to choose the distrubution of the conditional probability of the data given the hypothesis. That's why I am using it here: since there is no hyper parameter to tune it'll give a decent idea of the achievable accuracy
- NB works well with a hign number of features (that's why it's used with text classification) and we have 48 after processing. 
- NB works well with incomplete data. It's not our case here but if some informations were missing in some records (e.g. parents job type) it still would work (if smoothing is applied)

_Disadvantages:_
- NB relies on the strong assumption that features are totally independant

**Decision trees**

_General applications:_

Real life classification problems such as credit risk, rating (stars) classification, medical diagnosis

_Advantages:_
- They have explanatory power: since they are a sequence of decisions they are easy to understand and interpret. 
- It tells what features matter since they come up at the top of the tree.
- We don't know if our problem is linear, and DT don't require linearity
- It gives good results out of the box without too much parameter tuning (even if it may overfit). Therefore it gives an indication of the precision we can achieve in predictions for this problem.

_Disadvantages:_
It tends to overfit, but we can then tune hyperparameters such has the minimum sample split or max depth to overcome this problem.

**SVM**

_General applications_

Classification problems, such as  text categorization, image classification, bioinformatics (diagnosis)

_Advantages:_
- SVM works well with non linear classification problems, and we'll see that our problem is not, at least in its dimensional space.
- Although SVM can make data linearly separable by mapping them to a higher dimensional space, and thus
guarantee that every instance will be classified correctly (no training error). However doing so typically overfits. SVM allows for "soft margins": the analyst can decide that it's ok to have some of the data misclassified at the cost of a higher bias. I believe it's a good alternative option to decision trees since we're in a high dimensional and non linearly separable problem.

_Disadvantages:_
- SVM can be computationnally expansive but we don't have that many data and dimensions so it should not be an issue.
- SVM has only a 2 hyper parameters (with a 'rbf' kernel) so it's easy to tune with a grid search



### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))
    print""

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# we're gonna need to scale our data for SVM. Here's a function that scale all columns 
#of a panda data frame

from sklearn import preprocessing
def scaleDataFrame(df,scaler):
    df2 = df.copy();
    df2[df2.columns.values] = scaler.fit_transform(df2[df2.columns.values])
    return df2


In [8]:
from sklearn.externals.six import StringIO  
from IPython.display import Image
import pydot
from sklearn import tree
def exportTree(estimator,fileName):
    dot_data = StringIO() 
    tree.export_graphviz(estimator,out_file=dot_data,feature_names=list(X_all.columns.values),class_names=["failed","passed"],filled=True, rounded=True,special_characters=True) 
    graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
    if graph.write_pdf(fileName):
        print fileName," succesfully generated"

In [9]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn import svm

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = tree.DecisionTreeClassifier()
clf_C = clf = svm.SVC()

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]


# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C]:
  for size in [100, 200, 300]:
      train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)
      print '\n'

exportTree(clf_B,"first_decision_tree.pdf")
# Let's scale our data and train the SVM classifier again
# scaler = preprocessing.StandardScaler()
# X_train = scaleDataFrame(X_train,scaler)
# X_test = scaleDataFrame(X_test,scaler)

# X_train_300 = X_train[:300]
# y_train_300 = y_train[:300]
# train_predict(clf_A,X_train_300,y_train_300,X_test,y_test)



Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0009 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8550.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7481.



Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0010 seconds
Made predictions in 0.0007 seconds.
F1 score for training set: 0.8321.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7132.



Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0007 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8088.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.7500.



Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0008 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0001 seconds.
F1 score for test set: 0.6942.



Training a DecisionTreeClassifier using a training set s

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

##### ** Classifer 1 - GaussianNB**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0006           |          0.0007        |        0.8217    |     0.7481      |
| 200               |        0.0008           |          0.0004        |        0.8195    |     0.7132      |
| 300               |        0.0006           |          0.0003        |        0.8090    |     0.7500      |

** Classifer 2 - Decision tree**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0005           |          0.0002        |        1.0000    |     0.6667      |
| 200               |        0.0003           |          0.0002        |        1.0000    |     0.7077      |
| 300               |        0.0004           |          0.0004        |        1.0000    |     0.7333      |

** Classifer 3 - SVM**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0013           |           0.0010       |      0.9481      |     0.7838      |
| 200               |        0.0030           |           0.0019       |      0.9199      |     0.7755      |
| 300               |        0.0047           |           0.0016       |      0.9062      |     0.7586      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
At first sight GaussianNB seems better since it has the best high score on the biggest (most informed) training set. However we can't do much more with Naive Bayes, so let's see the two others where we still have fine tuning possibilities.

If SVM has a slightly better score, there is no clear winner yet. the marginal gain in accuracy is at the cost of 300% increase in computational cost (time). It feels reasonable to go on with both models and see which one performs best after a grid search. If SVM looks more promising based on the prelimiary results, I like the simplicity and explanatory power of decision trees as they implicitly explain which features are most important as they are closer to the root.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

#### SVM
SVM separates the students in 2 categories: those who pass and those who don't. Eventhough in reality those categories arent clearly separable because they are intricated (think about the Ying and Yang image: you cannot separate ying and yang), SVM has mathematical trick to draw a clear frontier. This frontier is drawn such as it maximizes its distance from the closest data points (instances in the training set). In a way, SVM looks mostly at the instances with opposite results that are the closest to each other, so it makes an extra effort separating samples that are more similar, hence its power.

#### Decision tree
A decision tree is a simple sequence of decisions leading to a result. To make a prediction on a sample, it takes one feature at a time and compares its value to a threshold. It allows easy to interpret predictions such as "if the students' failures are more than one, the travel time is more than an hour and the student has been absent more than 5 days then the student will fail"

**Answer: **

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

#### Let's first work on the decision tree
In this section we'll grid search the hyperparameters of the decision tree in order to find the tree that fits best our data and outputs the best f1 score.

In [10]:
from sklearn import grid_search

from sklearn.metrics import make_scorer


DTparams = {
    'criterion':['gini','entropy'],
    'splitter':['random','best'],
    'min_samples_split':list(2*i for i in range(1,50)),
    'max_depth':list(range(1,15))
}
f1_scorer = make_scorer(f1_score, pos_label="yes")
grid_searched_decision_tree = grid_search.GridSearchCV(tree.DecisionTreeClassifier(),param_grid=DTparams,scoring=f1_scorer,n_jobs=4,verbose=0);
grid_searched_decision_tree.fit(X_train_300,y_train_300);
pred = grid_searched_decision_tree.predict(X_test);
print f1_score(y_test,pred,pos_label="yes")
print grid_searched_decision_tree.best_estimator_

exportTree(grid_searched_decision_tree.best_estimator_,"grid_searched_decision_tree.pdf")

# Result:
# DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=2,
#             max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
#             min_samples_split=38, min_weight_fraction_leaf=0.0,
#             presort=False, random_state=None, splitter='random')
# 0.802919708029

0.794520547945
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=12, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='random')
grid_searched_decision_tree.pdf  succesfully generated


#### Let's now work on SVM
In this section we'll grid search the hyperparameters of the SVM classifier

In [11]:
#Let's scale our data and train the SVM classifier again
scaler = preprocessing.StandardScaler()
X_train_s = scaleDataFrame(X_train[:300],scaler)
X_test_s = scaleDataFrame(X_test,scaler)
X_train_s.to_csv("x_train_scaled_data.csv", sep='\t', encoding='utf-8')


gammas = list((10**exp for exp in range(-10, 10)))
C = list((10**exp for exp in range(-10, 10)))
SVMparams = {
    'kernel':['rbf'],
    'C':C,
    'gamma':gammas,
}
clf_gsvm = grid_search.GridSearchCV(svm.SVC(),param_grid=SVMparams,scoring=f1_scorer,n_jobs=4,verbose=0)
clf_gsvm.fit(X_train_s,y_train_300);
pred = clf_gsvm.predict(X_test_s);
print clf_gsvm.best_estimator_
print f1_score(y_test,pred,pos_label="yes")

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.789115646259


#### Remove outliers and compare results
If the data were normally distributed we could use an elliptic enveloppe to estimate which samples are outliers.
Instead we're gonna use OneClassSVM. It is primarily used to detect novelty: on a trained model free from outliers it draws a boundary such that it can estimate if a new instance belongs to that dataset. However, when the data distribution is not normal and operating is a relatively high dimensional space, OneClassSVM can be used as an outlier detection tool.
Here we have to reprocess the data from the beginning.

In [12]:
X_all2 = X_all.copy()
y_all2 = y_all.copy()

Now, we run the OneClassSVM to find the outliers. We run it on the training set only, so the test set remains untouched. The output is a vector of values either +1 or -1. -1 are outliers.
The number of outliers is quite high, about 25% and significantly decreases the size of our training set. It makes sense: the probability that a sample is an outlier in one dimension increases with the dimensionality.

In [13]:
# shuffle and split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_all2, y_all2, test_size=95, random_state=0)

# Run oneclass svm to detect outliers in testing set.
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
clf.fit(X_train2)
outliers = clf.predict(X_train2)

# putting everything together
X_train2['outliers'] = outliers
X_train2['passed'] = y_train2
#remove outliers
instances_before_outliers_removal = X_train2.shape[0]
X_train2 = X_train2[X_train2.outliers == 1]
instances_after_outliers_removal = X_train2.shape[0]
# remove the outlier column
X_train2 = X_train2.drop('outliers', 1)
# separate the results from the features
y_train2 = X_train2['passed']
X_train2 = X_train2.drop('passed', 1)

print "Training instances before outliers removal:",instances_before_outliers_removal
print "Training instances after outliers removal:",instances_after_outliers_removal
print "Instances in the labels of the training set:",y_train2.shape[0]
# print outliers


Training instances before outliers removal: 300
Training instances after outliers removal: 197
Instances in the labels of the training set: 197


/Users/nathanzylbersztejn/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nathanzylbersztejn/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Now we must remove the rows marked with -1 and process, shuffle-split and scale again.

Moment of truth: let's run our grid searches again

In [14]:
grid_searched_decision_tree_outliers_removed = grid_search.GridSearchCV(tree.DecisionTreeClassifier(),param_grid=DTparams,scoring=f1_scorer,n_jobs=4,verbose=0);
grid_searched_decision_tree_outliers_removed.fit(X_train2,y_train2);
pred = grid_searched_decision_tree_outliers_removed.predict(X_test2);
print f1_score(y_test2,pred,pos_label="yes")
print grid_searched_decision_tree_outliers_removed.best_estimator_

exportTree(grid_searched_decision_tree_outliers_removed.best_estimator_,"grid_searched_decision_tree_outliers_removed.pdf")

# 0.8
# DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
#             max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
#             min_samples_split=23, min_weight_fraction_leaf=0.0,
#             presort=False, random_state=None, splitter='random')



0.794520547945
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='random')
grid_searched_decision_tree_outliers_removed.pdf  succesfully generated


Now let's look at SVM results.

In [15]:
scaler = preprocessing.StandardScaler()
X_train2 = scaleDataFrame(X_train2,scaler)
X_test2 = scaleDataFrame(X_test2,scaler)
X_train2.to_csv("x_train2_scaled_data.csv", sep='\t', encoding='utf-8')

clf_gsvm3 = grid_search.GridSearchCV(svm.SVC(),param_grid=SVMparams,scoring=f1_scorer,n_jobs=4,verbose=0)
clf_gsvm3.fit(X_train2,y_train2);
pred = clf_gsvm3.predict(X_test2);
print clf_gsvm3.best_estimator_
print f1_score(y_test2,pred,pos_label="yes")

# SVC(C=12, cache_size=200, class_weight=None, coef0=0.0,
#   decision_function_shape=None, degree=3, gamma=0.0016, kernel='rbf',
#   max_iter=-1, probability=False, random_state=None, shrinking=True,
#   tol=0.001, verbose=False)
# 0.786666666667

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.794520547945


### Conclusion
Decision tree performs better than SVMs but to the margin only. However, they have 2 significant advantages: 
    1. They are super easy to interpret and to explain
    2. They run faster
I wasn't very succesful in improving the results by tuning the model. Removing outliers improves a tiny bit, but not significantly. I believe the reason is that the model mostly depend on a single feature (failures) as the decision tree suggests, so there is not much to tune. 

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*


**Answer: **
Final score on training set is 0.803.
It is clearly better than the untuned model (DT was 0.73 and SVM 0.76). The decision tree implies that the # of failures contains all of the explanatory power of the data. Looking at the CSV, the relationship is clearly visible when putting the "failures" and "passed" columns side by side. The decision tree suggest that no other variable can provide additional information to improve the classification. Either the rest of the data has no correlation at all with the labels, or they are all strongly correlated to "failures". Which makes sense: the reasons why a student failed in the past are likely to be similar to the reasons why the student is about to fail (not pass) this year. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.